# 

In [1]:
import os
import dotenv

from agents import Agent, Runner, trace, function_tool
from agents.mcp import MCPServerStreamableHttp

import chromadb
dotenv.load_dotenv()

True

In [2]:
# This is the same code as in the rag.ipynb notebook

chroma_client = chromadb.PersistentClient(path="../chroma")
nutrition_db = chroma_client.get_collection(name="nutrition_db")

@function_tool
def calorie_lookup_tool(query: str, max_results: int = 3) -> str:
    """
    Tool function for a RAG database to look up calorie information for specific food items, but not for meals.

    Args:
        query: The food item to look up.
        max_results: The maximum number of results to return.

    Returns:
        A string containing the nutrition information.
    """

    results = nutrition_db.query(
        query_texts=[query],
        n_results=max_results
    )
    
    if not results['documents'][0]:
        return f"No nutrition information found for: {query}"
    
    # Format results for the agent
    formatted_results = []
    for i, doc in enumerate(results['documents'][0]):
        metadata = results['metadatas'][0][i]
        food_item = metadata['food_item'].title()
        calories = metadata['calories_per_100g']
        category = metadata['food_category'].title()
        
        formatted_results.append(
            f"{food_item} ({category}): {calories} calories per 100g"
        )
    
    return "Nutrition Information:\n" + "\n".join(formatted_results)


In [33]:
exa_search_mcp = MCPServerStreamableHttp(
        name="Exa Search MCP",
        params={
            "url": f"https://mcp.exa.ai/mcp?{os.environ.get("EXA_API_KEY")}",
            "timeout": 30,
        },
        client_session_timeout_seconds=30,
        cache_tools_list=True,
        max_retry_attempts=1)

await exa_search_mcp.connect()

calorie_agent_with_search = Agent(
    name="Nutrition Assistant",
    instructions="""
    * You are a helpful nutrition assistant giving out calorie information.
    * You give concise answers.
    * You follow this workflow:  
        0) First, use the calorie_lookup_tool to get the calorie information of the ingredients. But only use the result if it's explicitly for the food requested in the query.
        1) If you couldn't find the exact match for the food or you need to look up the ingredients, search the EXA web to figure out the exact ingredients of the meal.
        Even if you have the calories in the web search response, you should still use the calorie_lookup_tool to get the calorie 
        information of the ingredients to make sure the information you provide is consistent.
        2) Then, if necessary, use the calorie_lookup_tool to get the calorie information of the ingredients.
    * Even if you know the recipe of the meal, always use Exa Search to find the exact recipe and ingredients. 
    * Once you know the ingredients, use the calorie_lookup_tool to get the calorie information of the individual ingredients.
    * If the query is about the meal, in your final output give a list of ingredients with their quantities and calories for a single serving. Also display the total calories.
    * Don't use the calorie_lookup_tool more than 10 times. 
    """,
    tools=[       
    calorie_lookup_tool],
    mcp_servers=[exa_search_mcp]
)


Reference query - shouldn't use ExaSearch

In [15]:
with trace("Nutrition Assistant with MCP - Only uses calorie_lookup_tool"):
    result = await Runner.run(calorie_agent_with_search, "How many calories are in total in a banana and an apple? Also give calories per 100g")
    print(result)

RunResult:
- Last agent: Agent(name="Nutrition Assistant", ...)
- Final output (str):
    - Per 100 g: banana 89 kcal, apple 52 kcal → combined 141 kcal per 200 g total.
    - Typical sizes (estimates):
      - 1 medium banana (~118 g): ~105 kcal
      - 1 medium apple (~182 g): ~95 kcal
      - Total for 1 banana + 1 apple: ~200 kcal
- 7 new item(s)
- 2 raw response(s)
- 0 input guardrail result(s)
- 0 output guardrail result(s)
(See `RunResult` for more details)


In [34]:
with trace("Nutrition Assistant with MCP"):
    result = await Runner.run(calorie_agent_with_search, "How many calories are in an english breakfast?")
    print(result.final_output)

Here’s a typical Full English Breakfast per serving (about 1 serving from a common recipe):

- Baked beans: 1/2 cup (100 g) – ~94 kcal
- Bacon (pork): 1 strip (30 g) – ~122 kcal
- Butter: 1 tablespoon (14 g) – ~100 kcal
- Cooking oil: 2 teaspoons (10 ml) – ~79 kcal
- Sliced bread: 1 slice (35 g) – ~88 kcal
- Pork sausage: 1 medium (60 g) – ~203 kcal
- Egg: 2 large (100 g) – ~194 kcal

Estimated total: ~880 kcal per serving (range ~850–990 kcal depending on brands/quantities).


In [ ]:
exa_search_mcp.cleanup()